# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [1]:
#! pip install pyLDAvis gensim spacy

### Import the libraries

In [69]:
from gensim.utils import simple_preprocess
from gensim.models import Phrases, LdaModel, CoherenceModel
from gensim.corpora import Dictionary
from pprint import pprint
from pyLDAvis import gensim_models
import pyLDAvis
import json
import typing
import spacy

In [3]:
en = spacy.load('en_core_web_sm')

### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

### Load the dataset

In [4]:
with open('newsgroups.json.txt') as file:
    data = json.load(file)

In [5]:
documents = list(data.get('content').values())

### Preprocess the data

### Email Removal

In [6]:
documents = [document.split(' ', 2)[2:][0] for document in documents]

In [7]:
documents[:5]

["(where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "(Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair numbe

### Newline Removal

In [8]:
documents = [document.replace('\n', ' ') for document in documents]

In [9]:
documents[:5]

["(where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15   I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is  all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.  Thanks, - IL    ---- brought to you by your neighborhood Lerxst ----     ",
 "(Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu  A fair number of brave souls who upgraded 

### Single Quotes Removal

In [10]:
documents = [document.replace('\'', '') for document in documents]

In [11]:
documents[:5]

['(wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15   I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is  all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.  Thanks, - IL    ---- brought to you by your neighborhood Lerxst ----     ',
 '(Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu  A fair number of brave souls who upgraded t

### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [12]:
documents_tokenized: typing.Generator = (simple_preprocess(document) for document in documents)

In [13]:
documents_tokenized = list(documents_tokenized)

### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [14]:
stopwords = list(en.Defaults.stop_words) + ['from', 'subject', 're', 'edu', 'use']

#### remove_stopwords( )

In [15]:
def remove_stopwords(texts):
    new_texts = []
    for text in texts:
        for word in text:
            if stopwords.__contains__(word):
                text.remove(word)
        new_texts.append(text)
    return new_texts

In [16]:
documents_stopwords_removed = remove_stopwords(documents_tokenized)


### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

In [17]:
phrases = Phrases(documents_stopwords_removed, threshold=100)

#### make_bigrams( )

In [20]:
def make_bigrams(texts):
    new_texts = []
    for text in texts:
        text = phrases[text]
        new_texts.append([
            f'{text[index]} {text[index + 1]}'
            for index in range(len(text)) if index < len(text) - 1
        ])
    return new_texts

In [21]:
documents_bigrams = make_bigrams(documents_stopwords_removed)

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [ ]:
#! python -m spacy download en

In [22]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#### lemmatizaton( )

In [23]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
documents_lemmatized = lemmatization(documents_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [25]:
print(documents_lemmatized[:1])

[['thing', 'thing', 'car', 'car', 'nntp_poste', 'nntp_poste', 'host', 'host', 'rac_wam', 'park', 'park', 'line', 'line', 'wondering_anyone', 'there', 'there', 'enlighten', 'enlighten', 'car', 'car', 'see', 'see', 'other', 'other', 'day', 'day', 'door', 'door', 'sport', 'sport', 'car', 'car', 'look', 'look', 'late', 'late', 'early', 'early', 'call', 'call', 'door', 'really', 'really', 'small', 'small', 'addition', 'addition', 'front', 'front', 'bumper', 'bumper', 'separate', 'separate', 'rest', 'rest', 'body', 'body', 'know', 'know', 'tellme', 'model', 'model', 'engine', 'engine', 'spec', 'spec', 'year', 'year', 'production', 'production', 'car', 'car', 'make', 'make', 'history', 'history', 'info', 'info', 'funky', 'funky', 'look', 'look', 'car', 'car', 'mail', 'mail', 'thank', 'bring', 'bring', 'neighborhood', 'neighborhood', 'lerxst']]


### Create a Dictionary

In [30]:
dictionary = Dictionary(documents_lemmatized)

### Filter low-frequency words

In [32]:
dictionary.filter_extremes(no_below=10, no_above=0.5)

### Create Corpus

In [36]:
corpus = [dictionary.doc2bow(text) for text in documents_lemmatized]

### Create Index 2 word dictionary

In [37]:
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [38]:
ldamodel = LdaModel(corpus,
                    num_topics=15,
                    id2word=id2word,
                    passes=20)

### Print the Keyword in the 10 topics

In [46]:
pprint(ldamodel.top_topics(corpus, topn=1)[:10])

[([(0.016763646, 'image')], nan),
 ([(0.023379896, 'gun')], nan),
 ([(0.9487834, 'ax')], nan),
 ([(0.026748477, 'window')], nan),
 ([(0.01307087, 'people')], nan),
 ([(0.117326476, 'com')], nan),
 ([(0.0100963265, 'people')], nan),
 ([(0.023528082, 'know')], nan),
 ([(0.036124248, 'key')], nan),
 ([(0.023091402, 'drive')], nan)]


/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [68]:
ldamodel.log_perplexity(corpus)

-7.1677076164075295

### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [71]:
coherence_model_lda = CoherenceModel(model=ldamodel, texts=documents_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_model_lda.get_coherence()

0.5698873966487777

/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [59]:
pyLDAvis.enable_notebook()

In [60]:
gensim_models.prepare(ldamodel, corpus, dictionary)

/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.083051 -0.116585       1        1  13.149329
4     -0.087534 -0.122970       2        1  11.456929
7     -0.083716 -0.024173       3        1   9.656299
11    -0.019662  0.124551       4        1   8.659054
12    -0.061678  0.010927       5        1   6.980717
3     -0.038664  0.203149       6        1   6.897900
9     -0.030108  0.190636       7        1   5.652332
2      0.563252 -0.044718       8        1   5.287528
1     -0.046785 -0.165217       9        1   5.236023
5     -0.013026 -0.003089      10        1   5.142631
14    -0.030395 -0.085376      11        1   5.055624
13    -0.038730 -0.201473      12        1   4.860644
8     -0.060353  0.021596      13        1   4.842274
0     -0.045046  0.111920      14        1   3.791075
10     0.075497  0.100821      15        1   3.331641, topic_info=         Term           Freq          Total Category  logprob  loglift
3559       ax  113491.000000  113491.000000  Default  30.0000  30.0000
178       com   16178.000000   16178.000000  Default  29.0000  29.0000
42    article   14604.000000   14604.000000  Default  28.0000  28.0000
653       key    3958.000000    3958.000000  Default  27.0000  27.0000
13       host    9289.000000    9289.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
44       base     315.556197    2788.050218  Topic15  -5.4758   1.2229
166      time     329.412951    9792.869526  Topic15  -5.4329   0.0096
1529    large     281.301133    1985.963878  Topic15  -5.5907   1.4473
460      work     299.923118    7671.207251  Topic15  -5.5266   0.1600
357     datum     267.015249    2758.208316  Topic15  -5.6429   1.0667

[1111 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
859       1  0.035783             _
859       8  0.049035             _
859      15  0.914444             _
5631      3  0.975432  abc_coverage
482       1  0.929928      absolute
...     ...       ...           ...
587      11  0.169986         young
587      12  0.056245         young
3634      8  0.970282            zi
6264      8  0.871920            zt
6264     10  0.113729            zt

[3376 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 8, 12, 13, 4, 10, 3, 2, 6, 15, 14, 9, 1, 11])

In [61]:
pyLDAvis.disable_notebook()
